In [2]:
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPooling2D,MaxPool2D,AveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint
from tensorflow.keras.regularizers import l2


img_height, img_width = 224, 224

data_dir = 'C:/Users/Shreyas/Downloads/data/data'




In [8]:
num_classes = 2 
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in vgg_base.layers:
  layer.trainable = False

x = vgg_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x) 
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=vgg_base.input, outputs=predictions)

opt = Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=15,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(data_dir + '/train',
                                                 target_size=(img_height, img_width),
                                                 batch_size=16,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_set = test_datagen.flow_from_directory(data_dir + '/test',
                                            target_size=(img_height, img_width),
                                            batch_size=16,
                                            class_mode='categorical',
                                            shuffle=False)

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

Found 297 images belonging to 2 classes.
Found 77 images belonging to 2 classes.


In [9]:
history = model.fit(training_set,
                    validation_data=test_set,
                    epochs=10,
                    callbacks=[early_stopping])



Epoch 1/10
19/19 [==============================] - 62s 3s/step - loss: 1.0521 - accuracy: 0.7205 - val_loss: 0.8254 - val_accuracy: 0.8442
Epoch 2/10
19/19 [==============================] - 64s 3s/step - loss: 0.7119 - accuracy: 0.8283 - val_loss: 0.8044 - val_accuracy: 0.7403
Epoch 3/10
19/19 [==============================] - 64s 3s/step - loss: 0.5640 - accuracy: 0.8586 - val_loss: 0.6092 - val_accuracy: 0.7662
Epoch 4/10
19/19 [==============================] - 64s 3s/step - loss: 0.4423 - accuracy: 0.9091 - val_loss: 0.5376 - val_accuracy: 0.7792
Epoch 5/10
19/19 [==============================] - 64s 3s/step - loss: 0.4188 - accuracy: 0.8956 - val_loss: 0.5233 - val_accuracy: 0.8312
Epoch 6/10
19/19 [==============================] - 63s 3s/step - loss: 0.4015 - accuracy: 0.9091 - val_loss: 0.4807 - val_accuracy: 0.8571
Epoch 7/10
19/19 [==============================] - 63s 3s/step - loss: 0.3371 - accuracy: 0.9394 - val_loss: 0.4698 - val_accuracy: 0.8571
Epoch 8/10
19/19 [==

In [10]:
from sklearn.metrics import classification_report
test_loss, test_accuracy = model.evaluate(test_set)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy * 100)

test_images, test_labels = next(test_set) 
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)
print(predicted_classes)

true_classes = np.argmax(test_labels, axis=1)
class_labels = ["laminar", "turbulent"]

print(classification_report(true_classes, predicted_classes, target_names=class_labels))

5/5 [==============================] - 9s 2s/step - loss: 0.4771 - accuracy: 0.8052
Test Loss: 0.47711899876594543
Test Accuracy: 80.51947951316833
1/1 [==============================] - 2s 2s/step
[0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0]
              precision    recall  f1-score   support

     laminar       1.00      0.62      0.77        16
   turbulent       0.00      0.00      0.00         0

    accuracy                           0.62        16
   macro avg       0.50      0.31      0.38        16
weighted avg       1.00      0.62      0.77        16



C:\Users\Shreyas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Shreyas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Shreyas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [11]:
#not a good architecture, in this case (with similar parameters)
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
for layer in resnet_base.layers:
  layer.trainable = False

x = resnet_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
resnet_predictions = Dense(num_classes, activation='softmax')(x)
resnet_model = Model(inputs=resnet_base.input, outputs=resnet_predictions)


In [12]:
resnet_opt = Adam(learning_rate=0.0001)
resnet_model.compile(optimizer=resnet_opt, loss='categorical_crossentropy', metrics=['accuracy'])
resnet_history = resnet_model.fit(training_set,
                    validation_data=test_set,
                    epochs=10,
                    callbacks=[early_stopping])

Epoch 1/10
19/19 [==============================] - 28s 1s/step - loss: 2.1243 - accuracy: 0.5017 - val_loss: 1.9007 - val_accuracy: 0.5195
Epoch 2/10
19/19 [==============================] - 24s 1s/step - loss: 1.9805 - accuracy: 0.5387 - val_loss: 1.7548 - val_accuracy: 0.5195
Epoch 3/10
19/19 [==============================] - 26s 1s/step - loss: 1.7968 - accuracy: 0.5623 - val_loss: 1.6217 - val_accuracy: 0.7792
Epoch 4/10
19/19 [==============================] - 26s 1s/step - loss: 1.6165 - accuracy: 0.6162 - val_loss: 1.5178 - val_accuracy: 0.6234
Epoch 5/10
19/19 [==============================] - 30s 2s/step - loss: 1.5428 - accuracy: 0.6061 - val_loss: 1.4064 - val_accuracy: 0.7792
Epoch 6/10
19/19 [==============================] - 29s 2s/step - loss: 1.4841 - accuracy: 0.5657 - val_loss: 1.3399 - val_accuracy: 0.6104
Epoch 7/10
19/19 [==============================] - 30s 2s/step - loss: 1.3169 - accuracy: 0.6330 - val_loss: 1.2490 - val_accuracy: 0.6753
Epoch 8/10
19/19 [==

In [13]:
test_loss, test_accuracy = resnet_model.evaluate(test_set)
print("ResNet50 Test Loss:", test_loss)
print("ResNet50 Test Accuracy:", test_accuracy * 100)


5/5 [==============================] - 4s 817ms/step - loss: 1.0697 - accuracy: 0.7273
ResNet50 Test Loss: 1.0697277784347534
ResNet50 Test Accuracy: 72.72727489471436
